# This Anime Does Not Exist - DeepDanbooru Editor 


### To run:
**Click "Open in Playground" above (if you see that option) and then Runtime > Run All and wait a few minutes for everything to load. (Takes about 4 minutes)**


---


### Credits

Customizable anime images using [aydao](https://twitter.com/aydaoAI)'s anime StyleGAN2 model from [TADNE](https://thisanimedoesnotexist.ai/).

Latent directions were discovered using [@Skyli0n](https://twitter.com/skyli0n)'s [DeepDanbooru](https://github.com/KichangKim/DeepDanbooru) notebook.


Notebook by [arfa](https://twitter.com/arfafax) and [@Skyli0n](https://twitter.com/skyli0n)


<div>
<a href="https://www.twitter.com/arfafax">
<img src="https://thisfursonadoesnotexist.com/arfa_sig.png" width="350"/>
</a>
</div>
<div>


## <- Click the triangle to view the code while you wait for it to load


In [ ]:
%tensorflow_version 1.x
!git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2 --depth 1

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1A-E_E32WAtTHRlOzjhhYhyyBDXLJN9_H', 'network-tadne.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=1x9cD_MV3UVboqnlpq9ZPWQ-kvFzxOGaz', 'deepdanbooru_dirs.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=1X2EEI9_bhAD8diPGB4n-JMxkFdG-AN2E', 'tadne-directions.zip', quiet=False)
%mkdir /content/directions
!unzip -o /content/tadne-directions.zip

In [ ]:
%cd /content/stylegan2

In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import tensorflow as tf
import tflex

In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'

In [ ]:
tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()
cores = tflex.get_cores()
tflex.set_override_cores(cores)
_G, _D, Gs = pickle.load(open("/content/network-tadne.pkl", "rb"))

In [ ]:
 def generate_image_from_w(w, truncation_psi):
    with tflex.device('/gpu:0'):
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
        Gs_kwargs.randomize_noise = False
        if truncation_psi is not None:
            Gs_kwargs.truncation_psi = truncation_psi
        synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation_psi, minibatch_size=1)
        images = Gs.components.synthesis.run(w, randomize_noise=False, **synthesis_kwargs)
        display(PIL.Image.fromarray(images[0], 'RGB').resize((500,500),Image.LANCZOS))

In [ ]:
import pickle
deepdanbooru_dirs = pickle.load(open('/content/deepdanbooru_dirs.pkl', 'rb'))

In [ ]:
named_directions = {}
latent_dirs = []
starts = []
ends = []

comp = 0
#TODO only use the most useful
for tag, vec in deepdanbooru_dirs.items():
    print(tag)
    for layers in [(0, 6), (6, 12), (12, 16)]:
        print(layers[0], layers[1])
        name = tag + "-" + str(layers[0]) + "-" + str(layers[1])
        named_directions[f'{name}'] = [comp, int(layers[0]), int(layers[1]), f'{name}']
    latent_dirs.append(np.matrix(vec[0]))
    comp = comp + 1
    #if comp > 50:
    #    break
named_directions

In [ ]:
len(named_directions)

In [ ]:
import yaml
%mkdir /content/animations
%cd /content/animations

# UI

In [ ]:
#@title Run UI (make sure you've done Runtime > Run All first or it won't work)
from ipywidgets import fixed
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
from IPython.display import clear_output
from ipywidgets import Layout, Button, Box, VBox, Label

box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='',
                    height="500px")
def normalize(v):
    norm=np.linalg.norm(v, ord=2)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm * len(v)

loaded_w = None
def display_sample(seed, truncation, direction, distance, scale, start, end, update, disp=True, save=None, noise_spec=None, **args):
    global loaded_w
    if update == False:
        print("False")
    # blockPrint()
    rng = np.random.RandomState(seed)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
      all_w = loaded_w.copy()
    
    for i, item in enumerate(args):
        value = args[item]
        start_l = named_directions[item][1]
        end_l = min(16, named_directions[item][2])
        direction_l = normalize(latent_dirs[named_directions[item][0]])
        for l in range(start_l, end_l):
            all_w[0][l] = all_w[0][l] + direction_l * value * scale

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

    generate_image_from_w(all_w, truncation)

seed = np.random.randint(0,100000)
style = {'description_width': 'initial', 'width': '150px'}
row_length = 6

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=18, step=1, value=18, description='end layer: ', continuous_update=False)


update = widgets.Checkbox(value=True, description="update")

directions_list = []
params = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}

for i, item in enumerate(named_directions):
    name = named_directions[item][3]
    widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
    directions_list.append(widget)
    params[item] = widget

top_box = widgets.HBox([seed, truncation])
#rename_dropdown = widgets.Dropdown(options=named_directions.keys())
#rename_box = widgets.Text()
#rename = widgets.Button(description="Rename")
#bot_box = widgets.HBox([rename_dropdown, rename_box, rename])

ui = widgets.VBox([top_box])

grid = widgets.GridspecLayout(len(directions_list)//3, 3, width='90%', height="300px")
for i in range(len(directions_list)//3):
    for j in range(3):
        grid[i, j] = directions_list[3*i+j]
    
ui2 = grid


random = widgets.Button(description="Randomize Sliders")
reset = widgets.Button(description="Reset Sliders")
def reset_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=0, description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//3, 3, width='90%', height="300px")
    for i in range(len(directions_list)//3):
        for j in range(3):
            grid[i, j] = directions_list[3*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    #last_button = mobile
    #if row_length == 1:
    #    last_button = desktop
    display(ui, out, ui2, reset, random, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

def random_sliders(b):
    directions_list = []
    params_new = {'seed': seed, 'truncation': truncation, 'direction': fixed(0), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer, 'update' : update}
    for i, item in enumerate(named_directions):
        name = named_directions[item][3]
        widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=np.random.normal(scale=2.5), description=name + ': ', continuous_update=False, style=style, layout={'width' : 'auto'})
        directions_list.append(widget)
        params_new[item] = widget
    params = params_new
    grid = widgets.GridspecLayout(len(directions_list)//3, 3, width='90%', height="300px")
    for i in range(len(directions_list)//3):
        for j in range(3):
            grid[i, j] = directions_list[3*i+j]
        
    ui2 = grid
    clear_output()
    out = widgets.interactive_output(display_sample, params)
    #last_button = mobile
    #if row_length == 1:
    #    last_button = desktop
    display(ui, out, ui2, reset, random, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))

save_name = widgets.Text()
load_name = widgets.Text()
save_btn = widgets.Button(description="Save")
load_btn = widgets.Button(description="Load")

def load_w(b):
    global loaded_w
    print("Loading ", load_name.value + ".npy")
    loaded_w = np.load(load_name.value + ".npy")

def save_w(b):
    global loaded_w
    rng = np.random.RandomState(params['seed'].value)
    z = rng.standard_normal(*Gs.input_shape[1:]).reshape(1, *Gs.input_shape[1:])

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.randomize_noise = False

    all_w = Gs.components.mapping.run(z, None, **Gs_kwargs) # [minibatch, layer, component]
    if loaded_w is not None:
        all_w = loaded_w.copy()

    for i, item in enumerate(params):
        if item not in ['seed', 'truncation', 'direction', 'distance', 'scale', 'start', 'end', 'update']:
            value = params[item].value
            start_l = named_directions[item][1]
            end_l = min(16, named_directions[item][2])
            direction_l = normalize(latent_dirs[named_directions[item][0]])
            for l in range(start_l, end_l):
                all_w[0][l] = all_w[0][l] + direction_l * value * params['scale'].value

    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * params['truncation'].value # [minibatch, layer, component]

    from datetime import datetime
    now = datetime.now()

    out_name = now.isoformat()
    
    if save_name.value:
        out_name = save_name.value
    np.save(out_name + ".npy", all_w)
    print("saved ", out_name + ".npy")

random.on_click(random_sliders)
reset.on_click(reset_sliders)
save_btn.on_click(save_w)
load_btn.on_click(load_w)
out = widgets.interactive_output(display_sample, params)
display(ui, out, ui2, reset, random, widgets.HBox([save_name, save_btn]), widgets.HBox([load_name, load_btn]))



# Animation stuff

In [ ]:
import math
from PIL import ImageFont
from PIL import ImageDraw
def interpolate_between_ws(seed_array, truncation=0.5, duration_sec = 12.0, smoothing_sec = 1.0, mp4_fps = 20, filename=None, text=False):
    #_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation, minibatch_size=8)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    #if seed_array[0] != seed_array[-1]:
    #    seed_array.append(seed_array[0])
    

    all_w = seed_array
        
    num_frames = int(np.rint(duration_sec * mp4_fps))
        
    def make_frame(t):
        blend = ((len(seed_array)-1)*t/duration_sec)%1.0
        src_i = math.floor((t/duration_sec)*(len(seed_array)-1))
        dst_i = src_i + 1
        #print(t, blend, src_i, dst_i)
        all_w_new = (blend * all_w[dst_i]) + (1 - blend) * all_w[src_i]
        all_images_src = Gs.components.synthesis.run(all_w_new, randomize_noise=False, **synthesis_kwargs)
        #all_images_dst = Gs.components.synthesis.run(all_w_dst, randomize_noise=False, **synthesis_kwargs)
        if text:
            new_im = PIL.Image.new('RGB', (512, 600))
            new_im.paste(PIL.Image.fromarray(np.median(all_images_src, axis=0).astype(np.uint8)), (0, 0))
            draw = ImageDraw.Draw(new_im)
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", size=16)
            draw.text((10, 512), "{:0.2f}".format((1-blend)), (255, 0, 0), font=font)
            draw.text((50, 512), str(seed_array[src_i]), (255, 255, 255), font=font)
            draw.text((10, 550), "{:0.2f}".format((blend)), (0, 255, 0), font=font)
            draw.text((50, 550), str(seed_array[dst_i]), (255, 255, 255), font=font)
            return np.array(new_im)
        else:
            return all_images_src[0]

    
    import moviepy.editor
    from datetime import datetime
    now = datetime.now()

    np.save(now.isoformat() + ".npy", all_w)
    mp4_file = '%s.mp4' % (now)
    if filename:
        mp4_file = filename
    mp4_codec = 'libx264'
    mp4_bitrate = '5M'

    video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
    video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)
    
    return mp4_file

In [ ]:
import glob
def preview_files():
    for file in sorted(glob.glob("*.npy")):
      try:
        truncation = 1
        print(file)
        w = np.load(file)
        noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
        Gs_kwargs = dnnlib.EasyDict()
        Gs_kwargs.randomize_noise = False


        if truncation != 1:
            w_avg = Gs.get_var('dlatent_avg')
            all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]

        generate_image_from_w(w, truncation)
      except:
        print("Error")

In [ ]:
from IPython.display import display, HTML

from IPython.display import HTML

def display_mp4(path):
    print(f'Read from {path}')
    from base64 import b64encode
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls loop autoplay>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))
    print('Display finished.')

# Generate animation

In [ ]:
!error Causing an error so the following cells are not run automatically....

Run this cell to preview all of the snapshots you've saved:

In [ ]:
preview_files()

List all of the files you want to animate between:

In [ ]:
ws = '''
pose1.npy
pose2.npy
pose3.npy
'''.split()
ws


Run this to generate the animation:

In [ ]:
w_arr = []
for w in ws:
    w_val = np.load(w)
    w_arr.append(w_val)

output_file = interpolate_between_ws(w_arr, duration_sec = 2.0, smoothing_sec = 1.0, mp4_fps = 30)

In [ ]:
display_mp4(output_file)

Run this to download the animation:

In [ ]:
from google.colab import files
files.download(output_file) 